# 爬蟲並將資料整理成dataframe

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
import requests
import time
import json
import pandas as pd
import psycopg2

In [ ]:
# 爬取Accupass網頁並用pandas整理並返回最終dataframe
def scrap_accupass():

    '''
    Part 1 用selenium在搜尋頁面中動態往下拉到最底，再爬取資訊
    '''
    option = webdriver.ChromeOptions()
    # option.add_argument("--no-sandbox")
    # option.add_argument("--headless")
    # option.add_argument("--disable-dev-shm-usage")
    # option.add_argument("--disable-gpu")
    option.add_argument("--window-size=1920,1080")
    option.add_experimental_option('excludeSwitches', ['enable-automation']) # 開發者模式。可以避開某些防爬機制，有開有保佑
    driver = webdriver.Chrome(options=option)
    # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)

    url = 'https://www.accupass.com/search?c=music&s=relevance'
    driver.get(url)
    driver.implicitly_wait(10) # 等待伺服器反應最多 10 秒，如果在時間到之前反應就提早結束等待

    # 捲動到底三次
    for i in range(10):
        driver.find_element('css selector', 'html').send_keys(Keys.END)
        time.sleep(3)

    # 找活動時間（使用者看的格式），並儲存至串列
    event_user_time = []

    event_div = driver.find_elements('css selector','#content > div > div.SearchPage-d3ff7972-container > main > section > div > div')
    for div in event_div:
        # 確保非空值
        if len(div.text.split('\n')) >=2:
            # 將日期.換成/顯示
            event_user_time.append(div.text.split('\n')[0].replace('.', '/'))

    # 找活動網址、活動ID並儲存至串列
    event_link = []
    event_id = []
    event_len = len(event_user_time)
    for i in range(1,event_len+1):
        # 活動網頁
        event_url = driver.find_element('css selector',f'#content > div > div.SearchPage-d3ff7972-container > main > section > div > div:nth-child({i}) > div > div > div > div > a')
        event_link.append(event_url.get_attribute('href'))
        # 活動ID（用來放在api link中）
        link = event_url.get_attribute('href')
        link_id = link.split('event/')[1].split('?')[0]
        event_id.append(link_id)

    driver.quit()

    '''
    Part 2 取到活動ID之後用requests,json進api爬取資訊
    '''
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

    # 用accupass本身的api取活動名稱、圖片網址、地址、表演者、展演空間、系統顯示時間
    event_name = []
    img_link = []
    address = []
    artists = []
    venue = []
    start_time = []
    end_time = []
    for ID in event_id:
        # print(ID)
        url = f"https://api.accupass.com/v3/events/{ID}"
        response = requests.get(url, headers=header)
        response.encoding = 'UTF-8' # 指定編碼方式，utf-8、UTF-8、UTF8、utf8 都一樣

        data = json.loads(response.text)
        event_name.append(data['title'])
        img_link.append(data['image200'])
        address.append(data['address'])
        venue.append(data['addressRemark'])
        start_time.append(data['eventTimeObj']['startDateTime'].split('T')[0])
        end_time.append(data['eventTimeObj']['endDateTime'].split('T')[0])

        guests_len = len(data['guests'])
        guests_per_event = []
        for i in range(guests_len):
            guests_per_event.append(data['guests'][i]['name'])
        # 若演出者為空值，則預設加入此字串
        if guests_per_event == []:
            artists.append('請參考活動網頁')
        elif guests_per_event != []:
            guests_per_event = '、'.join(guests_per_event)
            artists.append(guests_per_event)

    # 把各個資料list使用pandas存成dataframe
    accupass_df = pd.DataFrame(list(zip(event_name,event_user_time,venue,address,artists,img_link,event_link,start_time,end_time)), columns = ['EventName', 'EventTime', 'Venue', 'Address', 'Artists', 'ImageURL', 'PageURL', 'StartTime', 'EndTime'])
    # print(accupass_df)

    # 因為accupass音樂分類會有很多非演出活動，因此用關鍵字篩選活動名稱
    filter_name = ['演唱會','音樂會','音樂之夜','巡迴','獨奏會','演出','派對','歌謠祭','專場','巡演','Open Jam','演奏會','音樂夜']

    # 用活動名稱篩選出合適的活動，並存成filter dataframe （"|" means OR in pandas）
    filter_df = accupass_df.query(f'EventName.str.contains("|".join({filter_name}))', engine = 'python')

    # 有一個長期活動不符合，先找出他的index
    index_to_drop1 = filter_df[filter_df['EventName'] == '音樂進站-台北捷運音樂演出計畫 (邀你來表演)'].index
    index_to_drop2 = filter_df[filter_df['EventName'] == '【2024 存在音樂社 - 詞曲創作營】四天全方位創作課程 - 新生代導師 X 不插電演出  '].index

    # 再使用index把該筆資料從df移除
    filter_df = filter_df.drop(index=index_to_drop1)
    filter_df = filter_df.drop(index=index_to_drop2)

    return filter_df

df = scrap_accupass()


In [ ]:

# 匯出csv檔案，可在local檢查用
df.to_csv('accupass.csv',index=True)



# 連線DB並存進資料庫

In [ ]:
# 建table, 可跳過

# Postgres DB 連線
conn = psycopg2.connect(
    dbname='',
    user='',
    password='',
    host='',
    port='5432'
)

cursor = conn.cursor()

# Create table (if it doesn't exist)
create_table_query = '''
CREATE TABLE IF NOT EXISTS tb_kktix (
    id SERIAL PRIMARY KEY,
    EventName VARCHAR(255) NOT NULL,
    EventTime VARCHAR(255),
    Venue VARCHAR(255),
    Address VARCHAR(255),
    Artists VARCHAR(255),
    ImageURL VARCHAR(1000),
    PageURL VARCHAR(1000),
    StartTime TIMESTAMP,
    EndTime TIMESTAMP
);
'''
cursor.execute(create_table_query)
conn.commit()


In [ ]:
import pandas as pd
from sqlalchemy import create_engine


# PostgreSQL connection details
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'dpg--a.singapore-postgres.render.com'  # Replace with your endpoint
USER = ''  # Replace with your username
PASSWORD = ''  # Replace with your password
PORT = 5432  # Default PostgreSQL port
DATABASE = ''  # Replace with your database name

# Create SQLAlchemy engine
engine = create_engine(f'{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}')

# Insert the data into the PostgreSQL table
df.to_sql('tb_accupass', engine, if_exists='replace', index=False)

print("Data inserted successfully.")